In [15]:
import transformers
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

print(transformers.__version__)

4.25.1


In [2]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-uncased" 
batch_size = 32

In [3]:
from datasets import load_dataset
import torch
from tqdm.notebook import tqdm

#Separate in train and test datasets
data_files = {"train": "./datasets/train_ner.json",'validation': "./datasets/val_ner.json","test": "./datasets/test_ner.json"}

#load custom NER dataset using Huggingface datasets liberary
datasets = load_dataset('json', data_files=data_files)

#lets define label names
label_list = ['O','collections']

#Lets set the device to cpu
device =  torch.device('cuda')

Using custom data configuration default-d2b35189dcfaf439
Found cached dataset json (/home/ai/.cache/huggingface/datasets/json/default-d2b35189dcfaf439/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 500000
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 200000
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 225096
    })
})

In [6]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,267683,"[@Colby43764173, @alienfrens, awesome, I, love, it]","[0, 1, 0, 0, 0, 0]"
1,272020,"[Meebit, #17996, bought, for, 285, WETH, 365133, USD, on, Opensea, #Meebits, #MeebitsNFT, nn]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
2,145927,"[Punk, 2719, is, for, sale, for, 763Ξ, 116951, US, #cryptopunks, #punk2719, #punklisting, #2traits, #male, #chinstrap, #wildhair]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
3,31973,"[CloneX, #8778, sold, for, 800, WETH, 1272736n, nPreviously, sold, for, NAn#NFT, Collection, #CLONEXXTAKASHIMURAKAMI, nFloor, Price, 835, #ETHnn]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,454674,"[@statewarrior1, @rarible, @PublicaeOrg, @CiprianPater, STATE, ticks, those, boxes, and, moren, Metaverse, more, on, this, this, weekn, L1, blockchain, n, Nft, geographical, gamen, Nft, stable, coin, leveragen, #decentralized, social, media, appn, wallet, n, p2p, communication, n, world, currencynn]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,438101,"[@michaeljburry, Our, FREE, AIRDROP, is, LivennMint, at, the, link, belownn, nnTo, thank, our, community, we, are, givin, away, some, GOBLINTOWN, NFTsnFirst, come, first, serve, Good, LucknnEnds, in, 24h⏰]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6,244854,"[@tacocat5555, @Pixelmon, @BBRCOfficial, @AzukiOfficial, @KaijuKingz, @SappySealsNFT, @RENGAinc, @CyberBrokers, @doodles, Doodles, 2, is, for, the, masses, and, genesis, box, will, be, the, Gen, 1, items, of, the, many, Gens, after, release]","[0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,184720,"[@AshtonNFTs, @ALBJSXMSK, @BoredApeYC, @opensea, @CoinbaseNFT, @OthersideMeta, @AzukiOfficial, @doodles, @ethereum, @cryptopunksnfts, @moonbirds, Buy, #SPKY, today, before, the, price, explodes, #SpookyShiba, is, holding, no, prisoners, with, the, way, they, move, forward, Big, things, coming, n, #BNB, #BSC, #P2E, #BTC, #cryptocurrecy, #AMAs, #nft, #merchandise, n]","[0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
8,129419,"[@YammoYammo, @mycilroos, @Deadfellaz, @BeVEENFT, Can, I, have, the, hoodie, too]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
9,51475,"[@ryeont, @BoredApeYC, @JiaHo, @baycsg, MINT, YOUR, NFT, @sikorr, @doodles, @yugalabs, @GordonGoner, @boredapebot, @CryptoGarga, @caiovicentino, @machibigbrother, @indigoherz, @JRNYclub, @mrghostinvblog, @cryptopunksnfts, @superplastic, @nullinger, @NFTKingz, @Harryleemedia, @veratheape, @888NEKOCLUB, @cmttat]","[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [8]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(int(label[word_idx]))
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(int(label[word_idx]) if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [9]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 4231, 12887, 5690, 2012, 2305, 102], [101, 2065, 2017, 2123, 2102, 3582, 26927, 2115, 2063, 4394, 2041, 2028, 1997, 1996, 22147, 3367, 4773, 2509, 18288, 19000, 2015, 2041, 2045, 2016, 2015, 1996, 6515, 21505, 1997, 1030, 19091, 13390, 2080, 1998, 2028, 1997, 2256, 4231, 12887, 17961, 18934, 102], [101, 1030, 11471, 24065, 2100, 2278, 1030, 2474, 26585, 7875, 2015, 102], [101, 2045, 2024, 2061, 2116, 8235, 4784, 1998, 8474, 2008, 3202, 2131, 10036, 6675, 2043, 1996, 3247, 2000, 4888, 1037, 1052, 22540, 2003, 2081, 2115, 2063, 2085, 6637, 2114, 3016, 2278, 20160, 27822, 4231, 12887, 7196, 2015, 4385, 3272, 3071, 2003, 13599, 2017, 2000, 1996, 2060, 6694, 6721, 1052, 22540, 9010, 2008, 4148, 2296, 2733, 102], [101, 2045, 2024, 2061, 2116, 8235, 4784, 1998, 8474, 2008, 3202, 2131, 10036, 6675, 2043, 1996, 3247, 2000, 4888, 1037, 1052, 22540, 2003, 2081, 2115, 2063, 2085, 6637, 2114, 3016, 2278, 20160, 27822, 4231, 12887, 7196, 2015, 4385, 3272, 3071, 2003, 13599, 2017

In [10]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/500 [00:00<?, ?ba/s]

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/226 [00:00<?, ?ba/s]

In [11]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [12]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [18]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
    save_strategy = 'epoch'
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [19]:
import numpy as np


metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [21]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/home/ai/anaconda3/envs/token/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 500000
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 156250
  Number of trainable parameters = 108893186
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using 

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results